# Importando as bibliotecas





In [1]:
import tensorflow as tf
tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
tf.config.list_physical_devices('GPU')


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4091289943419823830
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2252026676
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17073572025521127408
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
gpus = tf.config.list_logical_devices('GPU')

type(gpus)
if gpus:
  # Replicate your computation on multiple GPUs
  c = []
  for gpu in gpus:
    with tf.device(gpu.name):
        print(gpu.name)
        pass
gpu = gpus[0]

/device:GPU:0


In [4]:
import sys
import os
from sklearn.model_selection import train_test_split
sys.path.append('./')
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import ThresholdedReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import models
import h5py
from tensorflow.keras.models import Model
from keras import Input
from tensorflow.keras import optimizers
import json
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import sklearn.metrics as sklm
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit


In [5]:
CHARNN_DATA_CONFIG = "D:/SiDi/Project/Modulo II/github/multimodal-sentiment-analysis/rbsa2/charnn_cnn/charcnn_data_config.json"
charcnn_data_config = pd.read_json(CHARNN_DATA_CONFIG)

CNN_DATA_CONFIG = "D:/SiDi/Project/Modulo II/github/multimodal-sentiment-analysis/rbsa2/charnn_cnn/nn_config.json"
nn_data_config = pd.read_json(CNN_DATA_CONFIG)


In [6]:
#data_config = json.load(open("charcnn_data_config.json")) #Leitura do arquivo de configuracao
data_config = charcnn_data_config #Leitura do arquivo de configuracao

#nn_config = json.load(open("nn_config.json")) # Leitura de arquivo de configuracao, quais sao os parametros
nn_config = nn_data_config # Leitura de arquivo de configuracao, quais sao os parametros

alphabet_size = data_config["data"]["alphabet_size"]+1 #tamanho do alfabeto
input_size = data_config["data"]["input_size"] #Tamanho do input de zi

fully_connected_layers = nn_config["char_cnn_zhang"]["fully_connected_layers"] #Camadas Conectadas
conv_layers = nn_config["char_cnn_zhang"]["conv_layers"] #Camadas Convulacionasi
threshold = nn_config["char_cnn_zhang"]["threshold"] # Limiar
dropout_p = nn_config["char_cnn_zhang"]["dropout_p"] #Taxa de Dropout
embedding_size = nn_config["char_cnn_zhang"]["embedding_size"] # Tamnho do Embedding

In [7]:
DATASET_PATH_URL  = "D:/SiDi/Project/Modulo II/dataset/dataset_sidi_512.csv"
df = pd.read_csv(DATASET_PATH_URL, sep='\t')

print(len(df))
df.reset_index(inplace=True)

12000


In [117]:
df[0:1]

,index,Unnamed: 0,Unnamed: 0.1,image_name,NEG,NEU,POS,text,sent_text,sent_image,image_path,image_height,image_width
0,0,382583,382583,798338609870872577-2,0.774375,0.174701,0.050924,what does it look like i do for a living? (cra...,NEG,0,data/79833/798338609870872577-2.jpg,1280,722


In [116]:
df[8001:8002]

,index,Unnamed: 0,Unnamed: 0.1,image_name,NEG,NEU,POS,text,sent_text,sent_image,image_path,image_height,image_width
8001,8001,84282,84282,769436550190948352-1,0.016399,0.943351,0.04025,RT @NoHoesAdam: Thinking about how many times ...,NEU,1,data/76943/769436550190948352-1.jpg,1138,960


In [115]:
df[4001:4002]

,index,Unnamed: 0,Unnamed: 0.1,image_name,NEG,NEU,POS,text,sent_text,sent_image,image_path,image_height,image_width
4001,4001,261866,261866,783159977338736640-1,0.013936,0.054167,0.931898,Keystagram: Umm superfreak sounds good like me...,POS,2,data/78315/783159977338736640-1.jpg,936,750


,index,Unnamed: 0,Unnamed: 0.1,image_name,NEG,NEU,POS,text,sent_text,sent_image,image_path,image_height,image_width
0,0,382583,382583,798338609870872577-2,0.774375,0.174701,0.050924,what does it look like i do for a living? (cra...,NEG,0,data/79833/798338609870872577-2.jpg,1280,722


In [162]:
df.groupby(['sent_image']).get_group(0)[0:1000]

,index,Unnamed: 0,Unnamed: 0.1,image_name,NEG,NEU,POS,text,sent_text,sent_image,image_path,image_height,image_width
0,0,382583,382583,798338609870872577-2,0.774375,0.174701,0.050924,what does it look like i do for a living? (cra...,NEG,0,data/79833/798338609870872577-2.jpg,1280,722
1,1,443004,443004,802556641057054721-1,0.540009,0.304103,0.155888,No cheat just skill. #ClikerHeroes https://t.c...,NEG,0,data/80255/802556641057054721-1.jpg,707,1366
2,2,348885,348885,796032212407721984-1,0.513661,0.322456,0.163883,@KEILOin_DaTrunk I deleted https://t.co/qIhBkn...,NEG,0,data/79603/796032212407721984-1.jpg,1334,750
3,3,377487,377487,798011160532381696-1,0.712648,0.254173,0.033180,"""RT NYTFashion: How Nasty Gal went from an eBa...",NEG,0,data/79801/798011160532381696-1.jpg,561,1000
4,4,423703,423703,801181521679790080-1,0.713677,0.186854,0.099468,I hate this nigga lmaooo https://t.co/gPcyJfJESN,NEG,0,data/80118/801181521679790080-1.jpg,1334,750
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,366928,366928,797124816012791809-3,0.734435,0.182342,0.083223,What about this one? Is she worth a fuck? http...,NEG,0,data/79712/797124816012791809-3.jpg,1024,575
996,996,388716,388716,798744236803899392-1,0.750881,0.104132,0.144987,I swear. Listen to it. If you wanna be in a sa...,NEG,0,data/79874/798744236803899392-1.jpg,1024,576
997,997,409502,409502,800191074538901504-1,0.819542,0.138484,0.041973,I don't remember taking this photo but I proba...,NEG,0,data/80019/800191074538901504-1.jpg,2048,1152
998,998,305785,305785,784074855704559616-1,0.926570,0.045142,0.028288,@SVS_Sound Y'all know that you're mad geniuses...,NEG,0,data/78407/784074855704559616-1.jpg,518,598


In [144]:
df.groupby(['sent_image']).head()[df['sent_image'] == 0 ]

C:\Users\rbsa2\AppData\Local\Temp/ipykernel_7472/2168241912.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.groupby(['sent_image']).head()[df['sent_image'] == 0 ]


,index,Unnamed: 0,Unnamed: 0.1,image_name,NEG,NEU,POS,text,sent_text,sent_image,image_path,image_height,image_width
0,0,382583,382583,798338609870872577-2,0.774375,0.174701,0.050924,what does it look like i do for a living? (cra...,NEG,0,data/79833/798338609870872577-2.jpg,1280,722
1,1,443004,443004,802556641057054721-1,0.540009,0.304103,0.155888,No cheat just skill. #ClikerHeroes https://t.c...,NEG,0,data/80255/802556641057054721-1.jpg,707,1366
2,2,348885,348885,796032212407721984-1,0.513661,0.322456,0.163883,@KEILOin_DaTrunk I deleted https://t.co/qIhBkn...,NEG,0,data/79603/796032212407721984-1.jpg,1334,750
3,3,377487,377487,798011160532381696-1,0.712648,0.254173,0.033180,"""RT NYTFashion: How Nasty Gal went from an eBa...",NEG,0,data/79801/798011160532381696-1.jpg,561,1000
4,4,423703,423703,801181521679790080-1,0.713677,0.186854,0.099468,I hate this nigga lmaooo https://t.co/gPcyJfJESN,NEG,0,data/80118/801181521679790080-1.jpg,1334,750


In [146]:
df.resample("sent_image")["sent_image"].head()

ValueError: Invalid frequency: sent_image

In [9]:
# Verifica o balanceamento das classes
classes = df.groupby('sent_image')['index'].nunique()/df.shape[0] #Agrupe por index
classes

sent_image
0    0.333333
1    0.333333
2    0.333333
Name: index, dtype: float64

In [90]:
image_height = 128 #Defina o tamanho  da imagem
image_width = 128 #Defina a largura da iamge
from tensorflow.keras.preprocessing.image import load_img #carrega funcao para carraegar imagem
from tensorflow.keras.preprocessing.image import img_to_array #carrega funcao para converter imagem para array
from PIL import Image
#Funcao que remove quebra de linha
def remove_spaces(cell):  
    return cell.replace('\n',' ')
#Converte que carrega a image e depois convertte para array
def get_array(image_path):
    img = load_img("D:\\SiDi\\Project\\Modulo II\\dataset\\512" + '/' + f"{image_path}.jpg", target_size=(image_height, image_width), color_mode='grayscale')

    return img_to_array(img)


#Mapeaa os tipos de classes existente nos rotulos de predicoes
def to_one_hot(y, class_mapping, num_classes):
    y_int = y.map(class_mapping)
    return to_categorical(y_int, num_classes)

#X_text = df['text'].apply(remove_spaces)


# Numero de classes do dataset

In [12]:

num_classes = df['sent_text'].nunique()
#Faz o mapeeamento da classe com o id
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['sent_image']))}
class_mapping
#y_int = df['sent_text'].map(class_mapping)

{0: 0, 1: 1, 2: 2}

# Agrupamento e Separação por Rotulo da Classe (Sentimento da Imagem)

In [206]:
SLICING_SIZE = 1000
neg_df = df.groupby(['sent_image']).get_group(0).sample(n=SLICING_SIZE, ignore_index=True)
neu_df = df.groupby(['sent_image']).get_group(1).sample(n=SLICING_SIZE, ignore_index=True)

pos_df = df.groupby(['sent_image']).get_group(2).sample(n=SLICING_SIZE, ignore_index=True)

"""for id in neg_df.index[0:1]:
    print(id)
    test = neg_df.iloc[id].to_frame().T
test"""

0


,index,Unnamed: 0,Unnamed: 0.1,image_name,NEG,NEU,POS,text,sent_text,sent_image,image_path,image_height,image_width
0,1863,352154,352154,796184046242066433-2,0.932633,0.020901,0.046466,Can we have this wig back? (Also this was in m...,NEG,0,data/79618/796184046242066433-2.jpg,710,750


In [220]:
X_join_image = []
y_join = []

image_path = []
y_total = []



# Negative Class
for i in range(0,SLICING_SIZE, 1):
    # Negative Class
    X_image = neg_df.iloc[i].to_frame().T['image_name'].apply(get_array)
    y = to_one_hot(neg_df.iloc[i].to_frame().T['sent_image'], class_mapping, num_classes)
    X_join_image.extend(X_image)
    y_join.extend(y)

    # Positive Class
    X_image = neu_df.iloc[i].to_frame().T['image_name'].apply(get_array)
    y = to_one_hot(neu_df.iloc[i].to_frame().T['sent_image'], class_mapping, num_classes)
    X_join_image.extend(X_image)
    y_join.extend(y)
    
    # Neutro Class
    X_image = pos_df.iloc[i].to_frame().T['image_name'].apply(get_array)
    y = to_one_hot(pos_df.iloc[i].to_frame().T['sent_image'], class_mapping, num_classes)
    X_join_image.extend(X_image)
    y_join.extend(y)

#X_test_text = pd.Series(X_test_text)
X_join_image = pd.Series(X_join_image)
y_join = np.array(y_join)

In [97]:
"""X_join_image = []
y_join = []

image_path = []
y_total = []



# Negative Class
for i in range(1,1001,1):
    X_image = df[i-1:i]['image_name'].apply(get_array)
    y = to_one_hot(df[i-1:i]['sent_image'], class_mapping, num_classes)
    X_join_image.extend(X_image)
    y_join.extend(y)

# Positive Class
for i in range(4001,5001,1):
    X_image = df[i-1:i]['image_name'].apply(get_array)
    y = to_one_hot(df[i-1:i]['sent_image'], class_mapping, num_classes)
    X_join_image.extend(X_image)
    y_join.extend(y)

# Neutro Class
for i in range(8001,9001,1):
    X_image = df[i-1:i]['image_name'].apply(get_array)
    y = to_one_hot(df[i-1:i]['sent_image'], class_mapping, num_classes)
    X_join_image.extend(X_image)
    y_join.extend(y)

#X_test_text = pd.Series(X_test_text)
X_join_image = pd.Series(X_join_image)
y_join = np.array(y_join)"""

In [221]:
np.unique(y_join,  axis=0, return_counts=True)

(array([[0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.]], dtype=float32),
 array([1000, 1000, 1000], dtype=int64))

In [209]:
st =  StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1234)
for train_index, test_index in st.split(X_join_image, y_join):
    X_train_image, X_test_image = X_join_image.iloc[train_index],X_join_image.iloc[test_index]
    y_train, y_test = y_join[train_index], y_join[test_index]
    print(len(train_index), len(test_index))



2400 600


In [210]:
np.unique(y_train, axis=0, return_counts=True)

(array([[0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.]], dtype=float32),
 array([800, 800, 800], dtype=int64))

In [211]:
np.unique(y_test, axis=0, return_counts=True)

(array([[0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.]], dtype=float32),
 array([200, 200, 200], dtype=int64))

# Converte de Imagem para Array Numpy - image_to_numpy(image_df)

In [212]:
def image_to_numpy(image_df):
    all_images = []
    for i in range(len(image_df)):
        all_images.append(image_df.iloc[i])  
    return np.array(all_images)

X_train_image = image_to_numpy(X_train_image)
X_test_image = image_to_numpy(X_test_image)

In [257]:

X_join_image = image_to_numpy(X_join_image)

# (01) - Conjunto de Dados de Validacao - Declara a Classe ImageValidationCheckpoint

In [222]:
def predict(model, X_test_image, batch_size=32):
    y_predict = np.round(model.predict(X_test_image))
    return y_predict

def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

class ImageValidationCheckpoint(Callback):
    def __init__(self, filepath, X_test_image,y_test, metric = 'kappa'):
        self.X_test_image = X_test_image
        self.y_test = y_test
        self.metric = metric
        self.max_metric = float('-inf')
        self.max_metrics = None
        self.filepath = filepath
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        
        predicted_labels = predict(self.model, X_test_image)
        #true_labels = [label2Idx[x[1]] for x in self.test_data]
        true_labels = one_hot_decode(self.y_test)
        predicted_labels = one_hot_decode(predicted_labels)
        eval_metrics = {
            'accuracy' : sklm.accuracy_score(true_labels, predicted_labels),
            'f1_micro' : sklm.f1_score(true_labels, predicted_labels, average='micro'),
            'f1_macro' : sklm.f1_score(true_labels, predicted_labels, average='macro'),
            'f1_binary' : sklm.f1_score(true_labels, predicted_labels, average='weighted', pos_label = 1),
            'kappa' : sklm.cohen_kappa_score(true_labels, predicted_labels)
        }
        eval_metric = eval_metrics[self.metric]
        self.history.append(eval_metric)
        
        if epoch > -1 and eval_metric > self.max_metric:
            print("\n" + self.metric + " improvement: " + str(eval_metric) + " (before: " + str(self.max_metric) + "), saving to " + self.filepath)
            self.max_metric = eval_metric     # optimization target
            self.max_metrics = eval_metrics   # all metrics
            #self.model.save(self.filepath)

image_input = Input(shape=(image_height, image_width, 1), name='image_input', dtype='float32')
x = layers.Conv2D(32, (8, 8), activation='relu',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(32, (6, 6), activation='relu')(x)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(32, (8, 8), activation='relu',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Conv2D(32, (6, 6), activation='relu')(x)
x = layers.MaxPooling2D((6, 6))(x)
x = layers.Flatten()(x)
encoded_image = layers.Dense(256, activation='relu', name='encoded_image')(x)

output = layers.Dense(num_classes, activation='softmax', name='output')(encoded_image)
model_image = Model(image_input, output)
model_image.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-4),
                    metrics=['acc'])
model_image.summary()



Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image_input (InputLayer)    [(None, 128, 128, 1)]     0         
                                                                 
 conv2d_10 (Conv2D)          (None, 121, 121, 32)      2080      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 20, 20, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 15, 15, 32)        36896     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 2, 2, 32)         0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 128)               0   

C:\Users\rbsa2\multimodal-sentiment-analysis\multimodal-sentiment-analysis\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [223]:
n_repeats = 1
exp_history = []
optimize_for = 'accuracy'
model_file = 'v0.0.1_image-tweet-sent.h5'
with tf.device(gpu.name):
    for i in range(n_repeats):
        checkpoint = ImageValidationCheckpoint(model_file, X_test_image, y_test, metric=optimize_for)
        model_image.fit(X_train_image, y_train, epochs=10, batch_size=32,
                         validation_data=(X_test_image, y_test)
                         ,callbacks = [checkpoint])
        exp_history.append(checkpoint.max_metrics)
    


Epoch 1/10
74/75 [============================>.] - ETA: 0s - loss: 4.8211 - acc: 0.3323
accuracy improvement: 0.35333333333333333 (before: -inf), saving to v0.0.1_image-tweet-sent.h5
75/75 [==============================] - 3s 34ms/step - loss: 4.8034 - acc: 0.3321 - val_loss: 3.4122 - val_acc: 0.3550
Epoch 2/10
73/75 [============================>.] - ETA: 0s - loss: 2.8585 - acc: 0.3408
accuracy improvement: 0.36333333333333334 (before: 0.35333333333333333), saving to v0.0.1_image-tweet-sent.h5
75/75 [==============================] - 2s 29ms/step - loss: 2.8392 - acc: 0.3413 - val_loss: 2.1422 - val_acc: 0.3767
Epoch 3/10
75/75 [==============================] - 2s 28ms/step - loss: 2.0020 - acc: 0.3642 - val_loss: 1.6964 - val_acc: 0.3583
Epoch 4/10
75/75 [==============================] - 2s 29ms/step - loss: 1.5557 - acc: 0.3875 - val_loss: 1.5930 - val_acc: 0.36331 - acc: 0.387
Epoch 5/10
75/75 [==============================] - 2s 28ms/step - loss: 1.3379 - acc: 0.3996 - val_l

In [224]:
avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp_history]) / n_repeats
print(avg_result)
print(avg_acc)
for i, r in enumerate(exp_history):
    model_file = "ness_exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)


0.0625
0.375
0 0.0625 0.375 0.35946839434400824 ness_exp1_single-page_repeat-00.hdf5


[{'accuracy': 0.375,
  'f1_micro': 0.375,
  'f1_macro': 0.35946839434400824,
  'f1_binary': 0.3594683943440082,
  'kappa': 0.0625}]

In [216]:
model_image.save('v0.0.2_32-relu-bt4sa-128-image-tweet-sent.h5')

In [227]:
import time
start = time.time()
y_pred = model_image.predict(X_test_image)

print(time.time()-start)

0.2842388153076172


In [218]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index

pred_class = []
for i in range(len(y_pred)):
  pred = np.where(y_pred[i] == np.amax(y_pred[i]))
  pred_class.append(class_mapping[pred[0][0]])
    
print(classification_report(Y_test, pred_class))

              precision    recall  f1-score   support

           0       0.31      0.36      0.33       200
           1       0.31      0.28      0.29       200
           2       0.37      0.34      0.36       200

    accuracy                           0.33       600
   macro avg       0.33      0.33      0.33       600
weighted avg       0.33      0.33      0.33       600

